# Task
I have a CSV file named 'loan_predictions.csv' with features like Age, Experience, Income, Family, CCAvg, Education, Mortgage, Securities Account, CD Account, Online, and CreditCard, plus a target column called Actual_Status.

Please:

Load the file
Split the data into train and test sets (80/20)
Apply StandardScaler on all numerical features
Train a RandomForestClassifier
Predict on the test set and show confusion matrix + classification report
Inverse-transform Age, Income, and Education in the test set so they appear as real values
Add columns for Actual_Status and Predicted_Status
Export the clean test set (with real values and predictions) as a new CSV called 'loan_predictions_clean.csv' for Looker Studio
Show me the final exported DataFrame and the model metrics. also sheet name is "Data"

Here is all the data you need:
"Bank_Personal_Loan_Modelling.xlsx"

## Data loading

### Subtask:
Load the data from the "Bank_Personal_Loan_Modelling.xlsx" Excel file into a pandas DataFrame.


**Reasoning**:
Load the data from the "Bank_Personal_Loan_Modelling.xlsx" Excel file into a pandas DataFrame and display the first 5 rows.



In [ ]:
import pandas as pd

try:
    df = pd.read_excel('Bank_Personal_Loan_Modelling.xlsx', sheet_name='Data')
    display(df.head())
except FileNotFoundError:
    print("Error: 'Bank_Personal_Loan_Modelling.xlsx' not found.")
except Exception as e:
    print(f"An error occurred: {e}")

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


## Data splitting

### Subtask:
Split the data into training and testing sets.


**Reasoning**:
Split the data into training and testing sets using train_test_split, stratifying by the 'Personal Loan' column.



In [ ]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X = df.drop('Personal Loan', axis=1)
y = df['Personal Loan']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Data preparation

### Subtask:
Prepare numerical features for scaling.


**Reasoning**:
Create DataFrames `X_train_num` and `X_test_num` containing only the numerical features from `X_train` and `X_test`, respectively.



In [ ]:
# Numerical features to be scaled
numerical_features = ['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education', 'Mortgage']

# Create DataFrames for numerical features
X_train_num = X_train[numerical_features]
X_test_num = X_test[numerical_features]

## Feature engineering

### Subtask:
Scale numerical features using StandardScaler.


**Reasoning**:
Scale numerical features using StandardScaler and replace the original numerical columns in X_train and X_test with the scaled versions, preserving column names.



In [ ]:
from sklearn.preprocessing import StandardScaler

# Instantiate StandardScaler
scaler = StandardScaler()

# Fit on training data and transform training and testing data
X_train_num_scaled = scaler.fit_transform(X_train_num)
X_test_num_scaled = scaler.transform(X_test_num)

# Convert scaled arrays back to DataFrames with original column names
X_train_num_scaled = pd.DataFrame(X_train_num_scaled, columns=X_train_num.columns, index=X_train_num.index)
X_test_num_scaled = pd.DataFrame(X_test_num_scaled, columns=X_test_num.columns, index=X_test_num.index)

# Replace original numerical columns in X_train and X_test with scaled versions
X_train = X_train.drop(numerical_features, axis=1)
X_train = pd.concat([X_train, X_train_num_scaled], axis=1)

X_test = X_test.drop(numerical_features, axis=1)
X_test = pd.concat([X_test, X_test_num_scaled], axis=1)

## Model training

### Subtask:
Train a RandomForestClassifier model.


**Reasoning**:
Train a RandomForestClassifier model using the scaled training data and the corresponding target variable.



In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

## Model evaluation

### Subtask:
Evaluate the trained RandomForestClassifier model.


**Reasoning**:
Predict on the test set, generate the confusion matrix and classification report.



In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Predict on the test set
y_pred = rf_classifier.predict(X_test)

# Generate and print the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Generate and print the classification report
cr = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(cr)

Confusion Matrix:
[[901   3]
 [  6  90]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       904
           1       0.97      0.94      0.95        96

    accuracy                           0.99      1000
   macro avg       0.98      0.97      0.97      1000
weighted avg       0.99      0.99      0.99      1000



## Data wrangling

### Subtask:
Inverse transform the scaled 'Age', 'Income', and 'Education' columns in the test set.


**Reasoning**:
Inverse transform the scaled 'Age', 'Income', and 'Education' columns in X_test_num_scaled using the scaler fitted previously and update X_test with the inverse-transformed values.



In [ ]:
# Get the indices of the columns to inverse transform
indices_to_inverse = [X_train_num.columns.get_loc(col) for col in columns_to_inverse]

# Extract the relevant scales and means for the selected columns
scales_to_use = scaler.scale_[indices_to_inverse]
means_to_use = scaler.mean_[indices_to_inverse]

# Inverse transform the selected columns using the correct scales and means
X_test_num_inverse = X_test_num_scaled[columns_to_inverse].copy()  # Create a copy to avoid SettingWithCopyWarning

# Apply inverse transformation
X_test_num_inverse *= scales_to_use
X_test_num_inverse += means_to_use

# Update X_test with inverse-transformed values
X_test.loc[:, columns_to_inverse] = X_test_num_inverse

NameError: name 'columns_to_inverse' is not defined

**Reasoning**:
The error `ValueError: operands could not be broadcast together with shapes (1000,3) (7,) (1000,3)` suggests that the shapes of the arrays involved in the multiplication are incompatible.  The `scaler.scale_` attribute likely has a shape of (7,), corresponding to all 7 numerical features, while the input to `inverse_transform` has a shape of (1000, 3), which only contains 'Age', 'Income', and 'Education'.  We need to select the corresponding scales from the scaler object for the three columns instead of using all seven scales.  Let's correct the code to use the correct scales for the three columns and then try to inverse transform again.



In [ ]:
# Define the columns to inverse transform
columns_to_inverse = ['Age', 'Income', 'Education']

# Get the indices of the columns to inverse transform
indices_to_inverse = [X_train_num.columns.get_loc(col) for col in columns_to_inverse]

# Extract the relevant scales and means for the selected columns
scales_to_use = scaler.scale_[indices_to_inverse]
means_to_use = scaler.mean_[indices_to_inverse]

# Inverse transform the selected columns using the correct scales and means
X_test_num_inverse = X_test_num_scaled[columns_to_inverse].copy()  # Create a copy to avoid SettingWithCopyWarning

# Apply inverse transformation
X_test_num_inverse *= scales_to_use
X_test_num_inverse += means_to_use

X_test_num_inverse = pd.DataFrame(X_test_num_inverse, columns=columns_to_inverse, index=X_test_num_scaled.index)

# Update X_test with inverse-transformed values
X_test.loc[:, columns_to_inverse] = X_test_num_inverse
display(X_test.head())

,ID,ZIP Code,Securities Account,CD Account,Online,CreditCard,Age,Experience,Income,Family,CCAvg,Education,Mortgage
2388,2389,91768,1,0,0,0,64.0,1.635293,23.0,0.537809,-0.817695,1.0,-0.553909
2373,2374,94304,0,0,0,0,33.0,-0.973697,184.0,-0.332080,1.655325,2.0,-0.553909
4347,4348,90024,0,0,1,0,58.0,1.113495,22.0,0.537809,-0.990231,1.0,-0.553909
665,666,92866,1,0,1,0,54.0,0.330798,61.0,1.407697,0.044986,3.0,-0.553909
4182,4183,92691,0,0,0,1,55.0,0.765630,49.0,-0.332080,-0.645159,3.0,1.623398


## Data preparation

### Subtask:
Add 'Actual_Status' and 'Predicted_Status' columns to the test set.


**Reasoning**:
Concatenate X_test, y_test, and y_pred into a single DataFrame, rename the target column, and add a new column for predicted status.



In [ ]:
import pandas as pd

# Concatenate X_test, y_test, and y_pred
df_test = pd.concat([X_test, y_test, pd.Series(y_pred, name='Predicted_Status')], axis=1)

# Rename the 'Personal Loan' column to 'Actual_Status'
df_test = df_test.rename(columns={'Personal Loan': 'Actual_Status'})

# Display the head of the DataFrame to verify the changes
display(df_test.head())

,ID,ZIP Code,Securities Account,CD Account,Online,CreditCard,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Actual_Status,Predicted_Status
2388,2389.0,91768.0,1.0,0.0,0.0,0.0,64.0,1.635293,23.0,0.537809,-0.817695,1.0,-0.553909,0.0,NaN
2373,2374.0,94304.0,0.0,0.0,0.0,0.0,33.0,-0.973697,184.0,-0.332080,1.655325,2.0,-0.553909,1.0,NaN
4347,4348.0,90024.0,0.0,0.0,1.0,0.0,58.0,1.113495,22.0,0.537809,-0.990231,1.0,-0.553909,0.0,NaN
665,666.0,92866.0,1.0,0.0,1.0,0.0,54.0,0.330798,61.0,1.407697,0.044986,3.0,-0.553909,0.0,0.0
4182,4183.0,92691.0,0.0,0.0,0.0,1.0,55.0,0.765630,49.0,-0.332080,-0.645159,3.0,1.623398,0.0,NaN


**Reasoning**:
The previous code block did not correctly add the `Predicted_Status` column.  I will re-attempt the concatenation, ensuring correct data types and alignment.



In [ ]:
import pandas as pd
# Concatenate X_test, y_test, and y_pred, ensuring correct alignment
df_test = pd.concat([X_test, y_test, pd.Series(y_pred, name='Predicted_Status', index=X_test.index)], axis=1)

# Rename the 'Personal Loan' column to 'Actual_Status'
df_test = df_test.rename(columns={'Personal Loan': 'Actual_Status'})

# Display the head of the DataFrame
display(df_test.head())

,ID,ZIP Code,Securities Account,CD Account,Online,CreditCard,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Actual_Status,Predicted_Status
2388,2389,91768,1,0,0,0,64.0,1.635293,23.0,0.537809,-0.817695,1.0,-0.553909,0,0
2373,2374,94304,0,0,0,0,33.0,-0.973697,184.0,-0.332080,1.655325,2.0,-0.553909,1,1
4347,4348,90024,0,0,1,0,58.0,1.113495,22.0,0.537809,-0.990231,1.0,-0.553909,0,0
665,666,92866,1,0,1,0,54.0,0.330798,61.0,1.407697,0.044986,3.0,-0.553909,0,0
4182,4183,92691,0,0,0,1,55.0,0.765630,49.0,-0.332080,-0.645159,3.0,1.623398,0,0


## Data loading

### Subtask:
Export the processed test set DataFrame to a new CSV file named 'loan_predictions_clean.csv'.


**Reasoning**:
Export the `df_test` DataFrame to a CSV file named 'loan_predictions_clean.csv' without the index.



In [ ]:
df_test.to_csv('loan_predictions_clean.csv', index=False)

## Data exploration

### Subtask:
Display the final exported DataFrame and the model metrics.


**Reasoning**:
Load the saved dataframe and display the confusion matrix and classification report.



In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

# Load the dataframe
df_test = pd.read_csv('loan_predictions_clean.csv')

# Display first few rows of the dataframe
display(df_test.head())

# Assuming y_test and y_pred are available from the previous steps
# If not, you need to re-run the model evaluation

# Extract actual and predicted values
y_test = df_test['Actual_Status']
y_pred = df_test['Predicted_Status']

# Compute and print the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Compute and print the classification report
cr = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(cr)

,ID,ZIP Code,Securities Account,CD Account,Online,CreditCard,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Actual_Status,Predicted_Status
0,2389,91768,1,0,0,0,64.0,1.635293,23.0,0.537809,-0.817695,1.0,-0.553909,0,0
1,2374,94304,0,0,0,0,33.0,-0.973697,184.0,-0.332080,1.655325,2.0,-0.553909,1,1
2,4348,90024,0,0,1,0,58.0,1.113495,22.0,0.537809,-0.990231,1.0,-0.553909,0,0
3,666,92866,1,0,1,0,54.0,0.330798,61.0,1.407697,0.044986,3.0,-0.553909,0,0
4,4183,92691,0,0,0,1,55.0,0.765630,49.0,-0.332080,-0.645159,3.0,1.623398,0,0


Confusion Matrix:
[[901   3]
 [  6  90]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       904
           1       0.97      0.94      0.95        96

    accuracy                           0.99      1000
   macro avg       0.98      0.97      0.97      1000
weighted avg       0.99      0.99      0.99      1000



## Summary:

### Q&A

* **What is the overall accuracy of the trained RandomForestClassifier model?**  The model achieved an accuracy of 0.99, as shown in the classification report.

* **How many misclassifications did the model make (based on the confusion matrix)?** The confusion matrix shows 3 False Positives and 6 False Negatives, indicating a total of 9 misclassifications out of 1000 test samples.

* **Which columns were inverse-transformed and why?** The 'Age', 'Income', and 'Education' columns were inverse-transformed after scaling. This was done to represent these features in their original scale within the exported `df_test` DataFrame for visualization and analysis in Looker Studio, as requested by the user.  The original scaled values were not suitable for direct interpretation in a business context.

### Data Analysis Key Findings

* **High Model Accuracy:** The RandomForestClassifier achieved a high accuracy of 0.99, indicating strong predictive performance.
* **Low Misclassification Rate:**  The confusion matrix revealed only 9 misclassifications out of 1000 test samples (3 False Positives and 6 False Negatives).
* **Effective Feature Scaling:**  StandardScaler was successfully applied to numerical features, followed by inverse transformation of 'Age', 'Income', and 'Education' in the test set for better interpretability in Looker Studio.
* **Successful Data Export:** The processed test data, including predicted and actual loan statuses and inverse-transformed features, was exported to 'loan_predictions_clean.csv' for visualization.


### Insights or Next Steps

* **Investigate Misclassifications:**  Further analysis of the 9 misclassified instances could reveal patterns or biases in the data or model, leading to potential model improvements.
* **Feature Importance Analysis:** Analyze feature importance scores from the RandomForestClassifier to understand which features are most influential in predicting loan status.  This can guide further feature engineering or selection.


In [ ]:
df_test.to_csv('loan_predictions_clean.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

# Load the dataframe
df_test = pd.read_csv('loan_predictions_clean.csv')

# Extract actual and predicted values
y_test = df_test['Actual_Status']
y_pred = df_test['Predicted_Status']

# Generate classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Extract accuracy, recall, and F1-score for class 1 (loan accepted)
accuracy = report['accuracy']
recall_1 = report['1']['recall']
f1_score_1 = report['1']['f1-score']

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print(f"Recall (Class 1): {recall_1:.2f}")
print(f"F1-score (Class 1): {f1_score_1:.2f}")

Accuracy: 0.99
Recall (Class 1): 0.94
F1-score (Class 1): 0.95
